In [13]:
import csv
import sys
import os
import re
import time
from collections import Counter
import numpy as np

import plotly.plotly as py
import plotly
from plotly import tools
from plotly.offline import init_notebook_mode
from plotly.graph_objs import *
import plotly.tools as tls

sys.path.insert(0, '../')
from meta_data_helper import EventMetaDataHelper

plotly.tools.set_credentials_file(username='sehwchoi', api_key='Ze9cXbcfuewb3C8TLktc')
init_notebook_mode(connected=True)  

In [14]:
mention_path = "../../user_network_data/aggregated_mention_networks"
user_set_path = "../../user_network_data/Event Users"

incident_metadata_path = '../incident_metadata.csv'

event_helper = EventMetaDataHelper(incident_metadata_path)
events = event_helper.get_all_events()
events = list(events)
events.sort()

In [15]:
def __read_user_set(file_name):
    user_set = []
    with open(file_name) as file:
        user_set_data = csv.reader(file)
        for row in user_set_data:
            user_id = row[0]
            if user_id != "id":
                user_set.append(int(user_id))
    return user_set


In [16]:
user_set_by_event = {}
def read_user_set_for_all_events(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            file_match = re.search("(\d+)_", filename)
            if file_match:
                event_id = file_match.group(1)
                user_set = __read_user_set(os.path.join(root, filename))
                # print("process event: {} user_set_len: {}".format(event_id, len(user_set)))
                user_set_by_event[int(event_id)] = user_set
    print("user_set length: {}".format(len(user_set_by_event)))

read_user_set_for_all_events(user_set_path)
    

user_set length: 304


In [17]:
def count_unique_user2(file_name, event):
    usr_unique_mention_ct = {}
    user_set = user_set_by_event[event]
    
    if os.path.isfile(file_name):
        with open(file_name) as file:
            data = csv.reader(file)
            for pair in data:
                try:
                    user1 = int(pair[0])
                    user2 = int(pair[1])
                    # print("user1: {} user2: {}".format(user1, user2))
                    # check if user1 exist as a key and increment mention count1
                    if user1 not in usr_unique_mention_ct:
                        # count for an unique user2
                        count1 = 1
                        count2 = 0
                        usr_unique_mention_ct[user1] = [count1, count2]
                    else:
                        usr_unique_mention_ct[user1][0] +=1

                    # increment count2 if the user2 is in the set
                    if user2 in user_set:
                        #print("user2: {} in user_set".format(user2))
                        usr_unique_mention_ct[user1][1] +=1
                except Exception as e:
                    print("file: {} error: {}".format(file_name, e))
    else:
        print("file: {} not exist".format(file_name))
            
            #print("user1_mention: {}".format(usr_unique_mention_ct[user1]))           
    #print("usr_unique_mention_ct: {}".format(usr_unique_mention_ct))
    return usr_unique_mention_ct

In [18]:
def get_mention_fraction(mention_ct):
    print(type(mention_ct))
    count1_max = max(value[0] for value in mention_ct.values())
    count2_max = max(value[1] for value in mention_ct.values())
    mention_freq = []
    total_user1_num = len(mention_ct.keys())
    # print("total_user_num: {} count1_max: {} count2_max: {}".format(total_user1_num, count1_max, count2_max))
    for i in range(0, count2_max+1):
        freq = 0
        for counts in mention_ct.values():
            if counts[1] == i:
                freq += 1
        # print("num_seq: {} freq: {}".format(i, freq))
        mention_freq.insert(i, freq)
        
    mention_fraction = []
    for freq in mention_freq:
        mention_fraction.append(freq/total_user1_num)
    
    return mention_fraction
                

In [19]:
def get_user2_fraction(mention_ct):
    fraction_list = []
    for user in mention_ct:
        fraction_list.append(mention_ct[user][1] / mention_ct[user][0])
    
    # print("fraction_list: {}".format(fraction_list))
    
    cnt = Counter(round(x, 1) for x in fraction_list)
    # print("fraction count list: {}".format(cnt))
    
    x_user2_frac_seq = [round(x * 0.1, 1) for x in range(0, 10)]
    y_user1_frac = []
    total_user1_num = len(mention_ct.keys())
    for frac in x_user2_frac_seq:
        # print("frac: {} cnt: {}".format(frac, cnt[frac]))
        if frac in cnt:
            y_user1_frac.append(cnt[frac] / total_user1_num)
        else:
            y_user1_frac.append(0)

    # print("y_user1_frac: {}".format(y_user1_frac))
    return [x_user2_frac_seq, y_user1_frac]

file_name = os.path.join(mention_path, "{}_mention_network.csv".format(44))
mention_ct = count_unique_user2(file_name, 44)
graph2_data = get_user2_fraction(mention_ct) 

In [22]:
def graph_for_mention_frac(path, events, draw_graph):
    avg_user2_mentioned_list = []
    for event in events:
        file_name = os.path.join(path, "{}_mention_network.csv".format(event))
        mention_ct = count_unique_user2(file_name, event)
        if len(mention_ct) > 0:
            mention_frac = get_mention_fraction(mention_ct)
            graph2_data = get_user2_fraction(mention_ct) # Counter dictionary
            x_user2_frac_seq = graph2_data[0]
            y_user1_frac = graph2_data[1]

            print(mention_frac)
            cnt2_sum = 0
            for user1 in mention_ct:
                cnt2_sum += mention_ct[user1][1]
            avg = cnt2_sum / len(mention_ct.keys())
            print("event: {} avg_usr2_cnt: {}".format(event, avg))
            avg_user2_mentioned_list.append(avg)

            if draw_graph:
                trace1 = Bar(
                        x=['%s'%i for i in range(0, len(mention_frac)+1)],
                        y=mention_frac,
                        showlegend=False)

                trace2 = Bar(
                        x=['{} ~ {}'.format(x, round(x+0.1, 1)) for x in x_user2_frac_seq],
                        y=y_user1_frac,
                        showlegend=False)

                layout=Layout(
                        xaxis=XAxis(title='Num of unique user2'),
                        yaxis=YAxis(title='Fraction of user1'),
                        xaxis2=XAxis(title='Fraction of unique user2'),
                        yaxis2=YAxis(title='Fraction of user1'),
                        title='Event {} User Mention Network Fraction'.format(event))

                fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Plot 1', 'Plot 2'))
                fig.append_trace(trace1, 1, 1)
                fig.append_trace(trace2, 1, 2)

                fig['layout']['xaxis1'].update(title='Num of unique user2')
                fig['layout']['xaxis2'].update(title='Fraction of unique user2')
                fig['layout']['yaxis1'].update(title='Fraction of user1')
                fig['layout']['yaxis2'].update(title='Fraction of user1')

                fig['layout'].update(title='Event {} User1 Mentioning Unique Number of User2'.format(event))
                plotly.offline.iplot(fig, filename="my plot")

    avg_count = Counter(round(x, 1) for x in avg_user2_mentioned_list)
    print("avg_count: {}".format(avg_count))
    avg_max = max(avg_count.keys())
    print("avg_max: {}".format(avg_max))
    avg_seq = [round(x*0.1, 1) for x in range(0, int(avg_max*10)+1)]
    print("avg_seq: {}".format(avg_seq))
    y_seq = []
    for seq in avg_seq:
        cnt = 0
        if seq in avg_count:
            cnt = avg_count[seq]
        y_seq.append(cnt)
    trace_avg = Bar(
                x=avg_seq,
                y=y_seq,
                showlegend=False)
    layout=Layout(
                xaxis=XAxis(title='Avg unique user2 mentioned'),
                yaxis=YAxis(title='Number of events'),
                title='Avg number of unique user2 mentioned by user2')

    data = Data([trace_avg])
    fig = Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename="my plot")

    
graph_for_mention_frac(mention_path, events, True)

<class 'dict'>
[0.7680525164113785, 0.18818380743982493, 0.0262582056892779, 0.006564551422319475, 0.006564551422319475, 0.0, 0.002188183807439825, 0.0, 0.002188183807439825]
event: 42 avg_usr2_cnt: 0.3172866520787746
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



<class 'dict'>
[0.745136186770428, 0.19747081712062256, 0.04134241245136187, 0.009727626459143969, 0.003404669260700389, 0.00048638132295719845, 0.0019455252918287938, 0.0, 0.00048638132295719845]
event: 44 avg_usr2_cnt: 0.3409533073929961
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



file: ../../user_network_data/aggregated_mention_networks/45_mention_network.csv error: invalid literal for int() with base 10: '99CC33'
file: ../../user_network_data/aggregated_mention_networks/45_mention_network.csv error: invalid literal for int() with base 10: 'Mizz_Man'
file: ../../user_network_data/aggregated_mention_networks/45_mention_network.csv error: invalid literal for int() with base 10: "{u'description': {u'urls': []}}"
<class 'dict'>
[0.7475961538461539, 0.19064349112426035, 0.04012573964497041, 0.012943786982248521, 0.003883136094674556, 0.002403846153846154, 0.0011094674556213018, 0.0007396449704142012, 0.0001849112426035503, 0.0003698224852071006]
event: 45 avg_usr2_cnt: 0.35392011834319526
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



file: ../../user_network_data/aggregated_mention_networks/46_mention_network.csv error: invalid literal for int() with base 10: 'https://abs.twimg.com/images/themes/theme14/bg.gif'
<class 'dict'>
[0.7612492599171107, 0.19182948490230906, 0.036708111308466546, 0.006808762581409118, 0.0014801657785671995, 0.0007400828892835998, 0.0004440497335701599, 0.0005920663114268798, 0.0, 0.0, 0.00014801657785671994]
event: 46 avg_usr2_cnt: 0.30358200118413264
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



file: ../../user_network_data/aggregated_mention_networks/47_mention_network.csv error: invalid literal for int() with base 10: 'LoveLife instagram J__kell'
<class 'dict'>
[0.6960195686012898, 0.21114076050700467, 0.058038692461641096, 0.016788970424727597, 0.00778296642205915, 0.003669112741827885, 0.0020013342228152103, 0.00133422281521014, 0.0005559261730042251, 0.00066711140760507, 0.000333555703802535, 0.000333555703802535, 0.000333555703802535, 0.000111185234600845, 0.00022237046920169, 0.0, 0.000333555703802535, 0.000111185234600845, 0.000111185234600845, 0.000111185234600845]
event: 47 avg_usr2_cnt: 0.48576828997109184
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



...

<class 'dict'>
[0.8792926045016077, 0.05736334405144695, 0.026688102893890675, 0.014083601286173633, 0.008553054662379421, 0.0046302250803858525, 0.003215434083601286, 0.0019935691318327974, 0.0009646302250803858, 0.0006430868167202572, 0.0006430868167202572, 0.00045016077170418006, 0.00045016077170418006, 6.430868167202572e-05, 0.0003858520900321543, 0.00019292604501607716, 0.00012861736334405144, 6.430868167202572e-05, 6.430868167202572e-05, 0.0, 0.00012861736334405144]
event: 341 avg_usr2_cnt: 0.28990353697749194
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



<class 'dict'>
[0.9245035046728972, 0.04337032710280374, 0.014164719626168224, 0.007739485981308411, 0.0040887850467289715, 0.0018983644859813084, 0.0010221962616822429, 0.0010221962616822429, 0.0007301401869158878, 0.0007301401869158878, 0.00014602803738317756, 0.00014602803738317756, 0.00014602803738317756, 0.00014602803738317756, 0.00014602803738317756]
event: 342 avg_usr2_cnt: 0.15522780373831777
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



file: ../../user_network_data/aggregated_mention_networks/343_mention_network.csv error: invalid literal for int() with base 10: 'Mon Jan 26 01:19:10 +0000 2009'
file: ../../user_network_data/aggregated_mention_networks/343_mention_network.csv error: invalid literal for int() with base 10: 'Mon Jan 26 01:19:10 +0000 2009'
file: ../../user_network_data/aggregated_mention_networks/343_mention_network.csv error: invalid literal for int() with base 10: 'False'
<class 'dict'>
[0.8953959018466987, 0.07108525170756387, 0.016696180116367317, 0.006450796863141917, 0.004047558816089046, 0.002023779408044523, 0.0015178345560333923, 0.0006324310650139134, 0.0003794586390083481, 0.0005059448520111307, 0.00012648621300278268, 0.0, 0.0003794586390083481, 0.0006324310650139134, 0.0, 0.00012648621300278268]
event: 343 avg_usr2_cnt: 0.1871995952441184
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



file: ../../user_network_data/aggregated_mention_networks/345_mention_network.csv error: invalid literal for int() with base 10: 'es'
file: ../../user_network_data/aggregated_mention_networks/345_mention_network.csv error: invalid literal for int() with base 10: 'True'
<class 'dict'>
[0.8035961272475796, 0.17496542185338865, 0.017289073305670817, 0.002074688796680498, 0.0006915629322268327, 0.0013831258644536654]
event: 345 avg_usr2_cnt: 0.22544951590594745
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



avg_count: Counter({0.4: 31, 0.3: 20, 0.5: 19, 0.2: 13, 0.6: 11, 0.1: 11, 0.0: 11, 0.7: 8, 0.8: 6, 1.0: 2, 0.9: 2, 1.4: 1})
avg_max: 1.4
avg_seq: [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4]
